In [1]:
import pandas as pd

In [2]:
links = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/links.csv')
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [3]:
movies = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/movies.csv')

In [4]:
ratings = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/ratings.csv')


In [5]:
tags = pd.read_csv('/Users/gabrielwarner/Downloads/ml-latest-small/tags.csv')

In [6]:
ratings = ratings.drop('timestamp', axis=1)

In [7]:
ratings.shape

(100836, 3)

In [8]:
new = pd.merge(links, movies, on='movieId')

In [9]:
new_2 = pd.merge(new, ratings, on='movieId')

In [10]:
new_3 = pd.merge(new_2, tags, on='movieId')

In [11]:
# Check for null values
new_3.isnull().sum()

movieId      0
imdbId       0
tmdbId       0
title        0
genres       0
userId_x     0
rating       0
userId_y     0
tag          0
timestamp    0
dtype: int64

In [12]:
# replace | with , in the genres column
new_3['genres'] = new_3['genres'].str.replace('|',' , ')

In [13]:
# split genre column based on ,
new_3['new_genres'] = new_3['genres'].str.split(',', 10)

In [14]:
# explode new_genres
df = new_3.explode('new_genres')

In [15]:
# drop genres from df
df_2 = df.drop(columns=['genres'])

In [16]:
tags = tags.drop(columns=['timestamp'])

In [17]:
tags.head(1)

,userId,movieId,tag
0,2,60756,funny


In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [19]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(tags['tag'])

In [20]:
tfidf_matrix.shape

(3683, 2421)

In [21]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [22]:
new_3 = pd.merge(new_2, tags, on='movieId')

In [23]:
tags_movies = pd.merge(tags, movies, on='movieId')

In [24]:
tags_movies.head(1)

,userId,movieId,tag,title,genres
0,2,60756,funny,Step Brothers (2008),Comedy


In [25]:
tags_1 = tags_movies.drop(columns=['genres'])

In [26]:
# change tag to title and add title to tags df
tags_2 = tags_1.reset_index()
titles = tags_2['title']
indices = pd.Series(tags_2.index, index=tags_2['title'])

In [27]:
#def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

IndentationError: unexpected indent (<ipython-input-27-a1e6d1d0887e>, line 2)

In [30]:
tfidf = TfidfVectorizer(stop_words='english')
tags_1['tag'] = tags_1['tag'].fillna('')
tfidf_matrix = tfidf.fit_transform(df['tag'])
tfidf_matrix.shape

(765271, 1671)

In [31]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

RuntimeError: nnz of the result is too large

In [32]:
indices = pd.Series(df.index, index=df['title']).drop_duplicates()

In [33]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Sort the movies based on the similarity score
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]
    movie_indices = [i[0] for i in sim_scores]
    # Return the top 5 most similar movies
    movies=df['title'].iloc[movie_indices]
    imdb_id=df['imdb_id'].iloc[movie_indices]
    dict={"Movies":movies,"IMDB_id":imdb_id}
    final_df=pd.DataFrame(dict)
    final_df.reset_index(drop=True,inplace=True)
    return final_df

In [34]:
get_recommendations('Inception').head(10)

KeyError: 'Inception'